## Web scaping and cleaning
source = fantasy.premierleague/statistics

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd

import time
from fake_useragent import UserAgent

In [2]:
#lets open a web driver (Chrome) for selenium with a the url
url = 'https://fantasy.premierleague.com/statistics'
PATH = "/Applications/chromedriver" # path to the chromedriver executable
driver = webdriver.Chrome(PATH)
driver.get(url)

In [151]:
# we use the x_path feature after inspecting the website to find out desired button and clicking to reveal the players weekly stats
driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20)
                      .until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div[2]/div/div[1]/table/tbody/tr[1]/td[1]/button'))))

In [152]:
#Here I just scraped the table head for my column names late
[i.text for i in soup.find(class_="Table-ziussd-1 ElementDialog__HistoryTable-gmefnd-16 hraaSw").find('thead').find('tr').find_all('th')]

['GW',
 'OPP',
 'Pts',
 'MP',
 'GS',
 'A',
 'CS',
 'GC',
 'OG',
 'PS',
 'PM',
 'YC',
 'RC',
 'S',
 'B',
 'BPS',
 'I',
 'C',
 'T',
 'II',
 'NT',
 'SB',
 '£']

In [3]:
# I set up double for loops- The outer one for every page on the website (1-24) 
#and the inner one for every player on a given page (30 players per page)

p_dict={}
row_data = []
p_names = []
for i in range(1,25):
    for i in range(1,31):
        driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div[2]/div/div[1]/table/tbody/tr[{i}]/td[1]/button'))))
        
        time.sleep(3)
        soup_file=driver.page_source
        soup = BeautifulSoup(soup_file)

        player = soup.find(class_="ElementDialog__ElementHeading-gmefnd-2").text
        position = soup.find(class_ = 'ElementDialog__ElementTypeLabel-gmefnd-3').text
        team = soup.find(class_='ElementDialog__Club-gmefnd-4 bIUoFS').text
        row_data = [player,position, team]
        p_names.append(player)
        p_dict[player] = row_data[1:]
        for row in soup.find(class_="Table-ziussd-1 ElementDialog__HistoryTable-gmefnd-16 hraaSw").find('tbody').find_all('tr')[:-1]:
           p_dict[player].append([i.text for i in row.find_all('td')])

        close_button = driver.find_element_by_xpath(f'//*[@id="root-dialog"]/div/dialog/div/div[1]/div/button')    
        close_button.click()
            
    next_pg_button = driver.find_element_by_xpath(f'//*[@id="root"]/div[2]/div/div[1]/div[3]/button[3]')
    next_pg_button.click()
    
    
 

KeyboardInterrupt: 

In [137]:
# putting the acquired data into a pandas datafraame

df = pd.DataFrame({}) # create an empty dict

for name in p_names:     
    #i = 1        
    lists = p_dict[name][2:]
    list1 = p_dict[name][:2]

    for list_ in lists:
        index_ = f'{name}'
        one_game_df = pd.DataFrame.from_dict({index_:list1+ list_}, orient = 'index')
        df = pd.concat([df, one_game_df], axis = 0)
columns = ['name','position','team','GW',
 'OPP',
 'Pts',
 'Mins.P',
 'Gls_S',
 'Assists',
 'Cln_Sht',
 'Gls_Csd',
 'Own_gls',
 'Pen_Sc',
 'Pen_Msd',
 'YC',
 'RC',
 'Saves',
 'Bonus',
 'BPS',
 'Inf',
 'Crt',
 'Thrt',
 'ICT',
 'Net_T',
 'SB',
 'price']        
df.columns = columns

In [24]:
df

,position,team,GW,OPP,Pts,Mins.P,Gls_S,Assists,Cln_Sht,Gls_Csd,...,Saves,Bonus,BPS,Inf,Crt,Thrt,ICT,Net_T,SB,price_£
name,,,,,,,,,,,,,,,,,,,,,
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,2,CRY (H) 1 - 3,2,90,0,0,0,3,...,0,0,13,25.6,78.4,25.0,12.9,451998,1623046,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,3,BHA (A) 2 - 3,12,90,1,1,0,2,...,0,3,40,60.6,40.8,19.0,12.0,-274258,1344788,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,4,TOT (H) 1 - 6,6,45,1,0,0,4,...,0,0,21,35.4,16.6,23.0,7.5,76237,1473422,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,5,NEW (A) 1 - 4,11,90,1,1,0,1,...,0,3,49,85.4,78.7,56.0,22.0,-1258,1508759,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,6,CHE (H) 0 - 0,3,90,0,0,1,0,...,0,0,14,18.6,60.9,16.0,9.6,85046,1609144,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kayne Ramsay,Defender,Southampton,28,BHA (H) 1 - 2,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,-24,6917,4.0
Kayne Ramsay,Defender,Southampton,30,BUR (H) 3 - 2,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,32,7107,4.0
Kayne Ramsay,Defender,Southampton,31,WBA (A) 3 - 0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,53,7262,4.0


In [26]:
df[df.index=='Bruno Miguel Borges Fernandes']

,position,team,GW,OPP,Pts,Mins.P,Gls_S,Assists,Cln_Sht,Gls_Csd,...,Saves,Bonus,BPS,Inf,Crt,Thrt,ICT,Net_T,SB,price_£
name,,,,,,,,,,,,,,,,,,,,,
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,2,CRY (H) 1 - 3,2,90,0,0,0,3,...,0,0,13,25.6,78.4,25.0,12.9,451998,1623046,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,3,BHA (A) 2 - 3,12,90,1,1,0,2,...,0,3,40,60.6,40.8,19.0,12.0,-274258,1344788,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,4,TOT (H) 1 - 6,6,45,1,0,0,4,...,0,0,21,35.4,16.6,23.0,7.5,76237,1473422,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,5,NEW (A) 1 - 4,11,90,1,1,0,1,...,0,3,49,85.4,78.7,56.0,22.0,-1258,1508759,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,6,CHE (H) 0 - 0,3,90,0,0,1,0,...,0,0,14,18.6,60.9,16.0,9.6,85046,1609144,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,7,ARS (H) 0 - 1,2,74,0,0,0,1,...,0,0,7,9.2,17.5,8.0,3.5,-17677,1601697,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,8,EVE (A) 1 - 3,17,90,2,1,0,1,...,0,3,51,93.6,21.2,58.0,17.3,-108368,1495550,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,9,WBA (H) 1 - 0,11,90,1,0,1,0,...,0,3,44,58.0,86.5,56.0,20.1,1033358,2644080,10.7
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,10,SOU (A) 2 - 3,10,90,1,1,0,2,...,0,0,26,35.6,20.1,54.0,11.0,241785,2832510,10.8


In [9]:
df

,position,team,GW,OPP,Pts,Mins.P,Gls_S,Assists,Cln_Sht,Gls_Csd,...,Saves,Bonus,BPS,Inf,Crt,Thrt,ICT,Net_T,SB,price
name,,,,,,,,,,,,,,,,,,,,,
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,2,CRY (H) 1 - 3,2,90,0,0,0,3,...,0,0,13,25.6,78.4,25.0,12.9,451998,1623046,£10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,3,BHA (A) 2 - 3,12,90,1,1,0,2,...,0,3,40,60.6,40.8,19.0,12.0,-274258,1344788,£10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,4,TOT (H) 1 - 6,6,45,1,0,0,4,...,0,0,21,35.4,16.6,23.0,7.5,76237,1473422,£10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,5,NEW (A) 1 - 4,11,90,1,1,0,1,...,0,3,49,85.4,78.7,56.0,22.0,-1258,1508759,£10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,6,CHE (H) 0 - 0,3,90,0,0,1,0,...,0,0,14,18.6,60.9,16.0,9.6,85046,1609144,£10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kayne Ramsay,Defender,Southampton,28,BHA (H) 1 - 2,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,-24,6917,£4.0
Kayne Ramsay,Defender,Southampton,30,BUR (H) 3 - 2,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,32,7107,£4.0
Kayne Ramsay,Defender,Southampton,31,WBA (A) 3 - 0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,53,7262,£4.0


In [14]:
df.rename({'price':'price_£'}, axis=1, inplace=True)

In [15]:
df['price_£'] = df['price_£'].str.replace('£','')
df["price_£"] = pd.to_numeric(df["price_£"])

In [16]:
df

,position,team,GW,OPP,Pts,Mins.P,Gls_S,Assists,Cln_Sht,Gls_Csd,...,Saves,Bonus,BPS,Inf,Crt,Thrt,ICT,Net_T,SB,price_£
name,,,,,,,,,,,,,,,,,,,,,
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,2,CRY (H) 1 - 3,2,90,0,0,0,3,...,0,0,13,25.6,78.4,25.0,12.9,451998,1623046,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,3,BHA (A) 2 - 3,12,90,1,1,0,2,...,0,3,40,60.6,40.8,19.0,12.0,-274258,1344788,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,4,TOT (H) 1 - 6,6,45,1,0,0,4,...,0,0,21,35.4,16.6,23.0,7.5,76237,1473422,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,5,NEW (A) 1 - 4,11,90,1,1,0,1,...,0,3,49,85.4,78.7,56.0,22.0,-1258,1508759,10.5
Bruno Miguel Borges Fernandes,Midfielder,Man Utd,6,CHE (H) 0 - 0,3,90,0,0,1,0,...,0,0,14,18.6,60.9,16.0,9.6,85046,1609144,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kayne Ramsay,Defender,Southampton,28,BHA (H) 1 - 2,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,-24,6917,4.0
Kayne Ramsay,Defender,Southampton,30,BUR (H) 3 - 2,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,32,7107,4.0
Kayne Ramsay,Defender,Southampton,31,WBA (A) 3 - 0,0,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,53,7262,4.0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21289 entries, Bruno Miguel Borges Fernandes to Kayne Ramsay
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   position  21289 non-null  object 
 1   team      21289 non-null  object 
 2   GW        21289 non-null  int64  
 3   OPP       21289 non-null  object 
 4   Pts       21289 non-null  int64  
 5   Mins.P    21289 non-null  int64  
 6   Gls_S     21289 non-null  int64  
 7   Assists   21289 non-null  int64  
 8   Cln_Sht   21289 non-null  int64  
 9   Gls_Csd   21289 non-null  int64  
 10  Own_gls   21289 non-null  int64  
 11  Pen_Sc    21289 non-null  int64  
 12  Pen_Msd   21289 non-null  int64  
 13  YC        21289 non-null  int64  
 14  RC        21289 non-null  int64  
 15  Saves     21289 non-null  int64  
 16  Bonus     21289 non-null  int64  
 17  BPS       21289 non-null  int64  
 18  Inf       21289 non-null  float64
 19  Crt       21289 non-null  float64
 20

In [21]:
# export gameweek 1 to 35 to scv
for i in range(1,34):
    df[df.GW==i].to_csv(f'data/GW{i}.csv')
    

In [22]:
pd.read_csv('data/GW1.csv')

,name,position,team,GW,OPP,Pts,Mins.P,Gls_S,Assists,Cln_Sht,...,Saves,Bonus,BPS,Inf,Crt,Thrt,ICT,Net_T,SB,price_£
0,Harry Kane,Forward,Spurs,1,EVE (H) 0 - 1,2,90,0,0,0,...,0,0,7,8.0,22.5,21.0,5.2,0,894557,10.5
1,Heung-Min Son,Midfielder,Spurs,1,EVE (H) 0 - 1,2,90,0,0,0,...,0,0,10,14.0,46.8,35.0,9.6,0,1724987,9.0
2,Mohamed Salah,Midfielder,Liverpool,1,LEE (H) 4 - 3,20,90,3,0,0,...,0,3,69,117.2,50.1,161.0,32.8,0,1883241,12.0
3,Patrick Bamford,Forward,Leeds,1,LIV (A) 4 - 3,7,61,1,0,0,...,0,1,28,40.0,0.5,27.0,6.8,0,226666,5.5
4,Jamie Vardy,Forward,Leicester,1,WBA (A) 0 - 3,13,90,2,0,1,...,0,3,52,71.2,10.3,36.0,11.8,0,1036141,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,Matija Sarkic,Goalkeeper,Wolves,1,SHU (A) 0 - 2,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,9380,4.5
419,Ryan Bennett,Defender,Wolves,1,SHU (A) 0 - 2,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,25352,4.5
420,Bruno André Cavaco Jordao,Midfielder,Wolves,1,SHU (A) 0 - 2,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,35928,4.5
421,Léo Bonatini,Forward,Wolves,1,SHU (A) 0 - 2,0,0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,0,18422,5.0
